**🎯 Key Features:**

Detect motion (moving vehicles)

Identify lane lines using Hough Transform

Highlight ROI (road region)

Add Gaussian noise to simulate real-world conditions

Overlay detection info with readable text annotations

All combined into a single annotated image output


**📦 Requirements**

Install these once (if you're using Google Colab):

In [ ]:
pip install gradio opencv-python-headless numpy pillow


In [ ]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image

def smart_surveillance(frame1, frame2, sigma=15):
    try:
        # Convert PIL images to OpenCV format
        frame1 = np.array(frame1.convert("RGB"))
        frame2 = np.array(frame2.convert("RGB"))
        img1 = cv2.resize(frame1, (640, 360))
        img2 = cv2.resize(frame2, (640, 360))

        # Grayscale conversion
        gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

        # Add Gaussian noise to frame1
        noise = np.random.normal(0, sigma, gray1.shape).astype(np.uint8)
        noisy1 = cv2.add(gray1, noise)

        # Frame differencing
        diff = cv2.absdiff(noisy1, gray2)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, np.ones((5, 5), np.uint8), iterations=2)

        # Motion detection via contours
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        output = img2.copy()
        motion_detected = False
        for cnt in contours:
            if cv2.contourArea(cnt) > 500:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(output, "Motion", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                motion_detected = True

        # Lane detection via Hough Transform
        edges = cv2.Canny(gray2, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 80, minLineLength=50, maxLineGap=20)
        if lines is not None:
            for line in lines[:10]:  # Draw top 10 lines
                x1, y1, x2, y2 = line[0]
                cv2.line(output, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(output, "Lanes Detected", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        if not motion_detected:
            cv2.putText(output, "No Motion Detected", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw ROI
        cv2.rectangle(output, (100, 250), (540, 330), (0, 255, 255), 2)
        cv2.putText(output, "ROI", (100, 245), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        return output

    except Exception as e:
        print("Processing error:", str(e))
        error_img = np.zeros((360, 640, 3), dtype=np.uint8)
        cv2.putText(error_img, "❌ Error during processing", (50, 180),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        return error_img

# Gradio Web Interface
gr.Interface(
    fn=smart_surveillance,
    inputs=[
        gr.Image(type="pil", label="Upload Frame 1"),
        gr.Image(type="pil", label="Upload Frame 2"),
        gr.Slider(0, 50, value=15, label="Gaussian Noise Sigma")
    ],
    outputs=gr.Image(label="🧠 Processed Surveillance Frame"),
    title="🚗 Motion & Lane Detection Tool",
    description="Detect motion and road lanes from two street view frames. Simulates noise and draws region of interest (ROI)."
).launch(debug=True)
